<a href="https://colab.research.google.com/github/olaviinha/NeuralAudioStemSeparation/blob/master/NeuralAudioStemSeparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Audio Stem Separation (22kHz sample rate)

This is an extension for [Deezer Spleeter](https://github.com/deezer/spleeter) to be run in Google Colab, using Google Drive as data storage.

`input` value may be
1.   Path to an individual audio file located in your Google Drive.
2.   Path to a directory in your Google Drive, containing multiple audio files. Each audio file found in the directory will be individually processed.
3.   Single YouTube URL
4.   Multiple YouTube URLs separated by spaces. Each youtube video will be individually processed.

Input will be separated to 4 stems: vocals, drums, bass and other. Stems will be saved as WAV files to your Google Drive (`output_dir`).

In [ ]:
#@title Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Settings

input = "ai/spleet/input/test.mp3" #@param {type:"string"}
output_dir = "ai/spleet/output" #@param {type:"string"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob

timestamp = datetime.today().strftime('%Y-%m-%d')

def stripPath(path, *args):
  if args:
    return path.strip("/")+"/"
  else:
    return path.strip("/")

def listAudio(dir):
  audiofiles = []
  for ext in ('*.wav', '*.aiff', '*.flac', '*.mp3', '*.ogg'):
    audiofiles.extend(glob(join(dir, ext)))
  return audiofiles

def basename(path):
  filename = os.path.basename(input).strip().replace(" ", "_")
  filebase = os.path.splitext(filename)[0]
  return filebase

tube_ind = "http"
tmp_dir = "/content/tmp/"
if not os.path.isdir(tmp_dir):
  !mkdir {tmp_dir}
drive_root = "/content/drive/My Drive/"
if tube_ind not in input:
  input = drive_root+stripPath(input)
output = drive_root+stripPath(output_dir, True)

### Install

In [ ]:
!pip install spleeter
if tube_ind in input:
  !pip install youtube-dl

In [ ]:
if tube_ind in input:
  !youtube-dl --restrict-filenames -x --no-continue --audio-format wav -o "{tmp_dir}%(title)s.%(ext)s" {input}
  tubeFiles = glob(tmp_dir+"/*.wav")

## Separate

In [ ]:
!gsutil -q -m cp -R gs://neural-research/olaviinha/splt-22khz-4st-a.json /content/cfg.json
if os.path.isdir(input) or tube_ind in input:
  if tube_ind in input:
    fileList = tubeFiles
  else:
    fileList = listAUdio(input)
  for file in fileList:
    #filename = basename(file)
    #file_output = output+filename
    #!mkdir file_output
    !spleeter separate -i "{file}" -o "{output}" -p /content/cfg.json
elif os.path.isfile(input):
  !spleeter separate -i "{input}" -o "{output}" -p /content/cfg.json
else:
  print('Unable to find input file/directory. Check input_dir')